In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from verbio import readers, features, settings, preprocessing, temporal, visualize, utils
import numpy as np
import pandas as pd
from scipy.stats import pearsonr

In [3]:
win_len = 15.0
win_stride = 5.0
eda_threshold = 2
hr_threshold = 0.2
pt_range = range(1, 74)

pt_names = [f'P{pt:03d}' for pt in pt_range]
sessions = ['TEST01','TEST02','TEST03','TEST04','TEST05','TEST06','TEST07','TEST08']

base_dir = '/home/jason/hubbs/project_verbio/data/raw/'
survey_dir = '/home/jason/hubbs/project_verbio/data/surveys/'

eda_fname = 'E4_EDA_PPT.xlsx'
hr_fname = 'E4_HR_PPT.xlsx'

In [12]:
all_triggers, all_scores = [], []

for pt in pt_names:
    triggers, survey_scores = [], []
    pt_valid = True
    for session in sessions:
        # Build paths to data
        survey_fname = f'{session}_afterPPT.xlsx'
        survey_path = os.path.join(survey_dir, session, survey_fname)
        eda_path = os.path.join(base_dir, pt, session, eda_fname)
        hr_path = os.path.join(base_dir, pt, session, hr_fname)
        # Skip session if missing
        if not os.path.exists(eda_path) or not os.path.exists(hr_path) or not os.path.exists(survey_path):
            pt_valid = False
            break
        
        # Read in dfs
        eda_df = readers.read_excel(eda_path)
        hr_df = readers.read_excel(hr_path)
        survey_df = pd.read_excel(survey_path, engine='openpyxl')
        # Convert EDA signals to numpy
        eda_signal = eda_df['EDA'].to_numpy()
        eda_times = eda_df[settings.time_key].to_numpy()
        # Get EDA features
        eda_fx = features.eda_features(
            signal=eda_signal, 
            times=eda_times, 
            sr=4, 
            win_len=win_len, 
            win_stride=win_stride
        )[['SCR_Peaks']]
        eda_fx = np.squeeze(eda_fx)
        # Convert HR signals to numpy
        hr_signal = hr_df['HR'].to_numpy()
        hr_times = hr_df[settings.time_key].to_numpy()
        # Window HR
        hr_fx = preprocessing.window_timed(
            x=np.gradient(hr_signal),
            times=hr_times,
            win_len=win_len,
            win_stride=win_stride,
            win_fn=lambda x: np.max(x)
        )
        hr_fx = np.array(hr_fx)
        # Parse survey score for anxiety
        survey_fx = survey_df[survey_df['PID'] == pt]['State Anxiety Enthusiasm Score'].iloc[0]
        # Count occurrences
        min_len = min(eda_fx.shape[0], hr_fx.shape[0])
        eda_fx = eda_fx[:min_len]
        hr_fx = hr_fx[:min_len]
        df = pd.DataFrame({'EDA':eda_fx,'HR':hr_fx})
        result_df = df[(df['HR'] > hr_threshold) | (df['EDA'] >= eda_threshold)]
        
        triggers.append(len(result_df)/len(df))
        survey_scores.append(survey_fx)
    
    if pt_valid:
        corr, p = pearsonr(triggers, survey_scores)
        print(f'Participant {pt} has correlation: {corr:.3f} | p-value: {p:.3f}')
        all_triggers += triggers
        all_scores += survey_scores

total_corr, total_p = pearsonr(all_triggers, all_scores)
print(f'Total corr: {total_corr:.3f} | Total p: {total_p:.3f}.')


Participant P004 has correlation: 0.270 | p-value: 0.518
Participant P005 has correlation: -0.243 | p-value: 0.561
Participant P008 has correlation: 0.190 | p-value: 0.653
Participant P016 has correlation: 0.104 | p-value: 0.806
Participant P020 has correlation: 0.526 | p-value: 0.180
Participant P021 has correlation: 0.270 | p-value: 0.517
Participant P023 has correlation: -0.666 | p-value: 0.071
Participant P032 has correlation: 0.455 | p-value: 0.257
Participant P035 has correlation: 0.163 | p-value: 0.700
Participant P037 has correlation: 0.506 | p-value: 0.200
Participant P039 has correlation: -0.267 | p-value: 0.522
Participant P041 has correlation: 0.507 | p-value: 0.199
Participant P042 has correlation: -0.557 | p-value: 0.151
Participant P044 has correlation: -0.294 | p-value: 0.479
Participant P047 has correlation: 0.817 | p-value: 0.013
Participant P050 has correlation: -0.567 | p-value: 0.143
Participant P051 has correlation: -0.237 | p-value: 0.571
Participant P053 has cor